In [ ]:
import multiprocessing
# multiprocessing.set_start_method('forkserver')
# from concurrent.futures import ProcessPoolExecutor
import controlpy
import time
import networkx as nx
import numpy as np
import csv
# import itertools
# import scipy
import numpy.linalg as LA
import scipy as sc
# from scipy import stats
from os import listdir
from os.path import join
# from igraph import  *
from scipy import stats
import matplotlib.pyplot as plt
from scipy import sparse
import sys
## from joblib import Parallel, delayed
from joblib import Parallel, delayed
# import tqdm
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.utils import shuffle
num_cores = multiprocessing.cpu_count()
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from sklearn.metrics import accuracy_score
import torch
import networkx as nx
import numpy as np
# from rdkit import Chem
import pickle
from torch_geometric.datasets import TUDataset
import matplotlib.colors as pltc
import random

In [ ]:
INCLUDE_GRM = True
INCLUDE_INVERSE_GRM = False #-----
INCLUDE_EIGEN_VALUES = True
INCLUDE_MIN_MAX = True
INCLUDE_MEAN = True
INCLUDE_LAP_SPECT = True
INCLUDE_LAP_SPECT_EXT = True
INCLUDE_METRIC_DIMENSION = True
INCLUDE_FEATURES = True
LAPLACIAN_ENERGY = True
ECC_SPECTRUM = True
ECC_ENERGY = True
WEINER_INDEX = True
TRACE_DEG_SEQ = True
INCLUDE_CYCLES = True

DESCRIPTOR_SIZE_ONE_LEADER = 5
LEADERS_SIZE_DET = 0.95

def add_type_attribute(G, node_types):
    type_mapping = {}
    for index, n in enumerate(G.nodes()):
        type_mapping[n] = node_types[index]
    nx.set_node_attributes(G, type_mapping,"type")
    return G

def get_leaders_followers(node_types, elem,G):
    G = add_type_attribute(G, node_types)
    if elem is 'C':
        leaders = [i for i,x in enumerate(node_types) if x !=elem]
        followers = [i for i,x in enumerate(node_types) if x ==elem]

    elif elem is 'VNode':
        attrib = nx.get_node_attributes(G, "type")
        leader_nodes = [x for x in G.nodes() if attrib[x]!='C']
        neighbors = []
        for n in leader_nodes:
            nei = list(G.neighbors(n))
            neighbors.extend(nei)
        neighbors = list(set(neighbors))        
        leaders = [i for i,x in enumerate(node_types) if x !='C']
        leaders.extend(neighbors)
        leaders = list(set(leaders))
        followers = [i for i in range(len(list(G.nodes()))) if i not in leaders]
        
        
    else:
        leaders = [i for i,x in enumerate(node_types) if x ==elem]
        followers = [i for i,x in enumerate(node_types) if x !=elem]
    return leaders, followers

def return_node_types(d,node_types):
    g = to_networkx(d, to_undirected = True)
    if node_types ==None:
        x = d.x[:,0].numpy().astype(int).tolist()
        return x,g
    x = d.x
    types_ = []
    for xv in x:
        type_index = (xv == 1.0).nonzero(as_tuple=True)[0].item()
        types_.append(node_types.get(type_index))
    return types_, g   

def get_descriptor(d,node_types,ALL_ELEMENTS):
    atom_types, G = return_node_types(d, node_types)
    
    
    if nx.is_connected(G)== False:
        G = add_vertex(G)
        atom_types.append("VNode")
    if G.number_of_nodes()<10:
        G,atom_types = add_multiple_copies(G,atom_types)
    A1 = -(np.matrix(nx.laplacian_matrix(G,nodelist=sorted(G.nodes())).todense()))
    n = A1.shape[0]
    desc = {}
    new_desc = []
    total_types = len( ALL_ELEMENTS ) 
    desc_size = (total_types * DESCRIPTOR_SIZE_ONE_LEADER) + 2
    
    leaders = []
    nodes = list(G.nodes())
    
    old_n = A1.shape[0]
    num_l_idx = 0
    
    for elem in ALL_ELEMENTS:
        traces = ranks = min_eigs = max_eigs = itraces = iranks = imin_eigs = imax_eigs = metric_dimension = 0
        leaders, follows = get_leaders_followers(atom_types, elem, G)
        if len(leaders) is 0 or (len(follows) is 0):
            new_desc.extend( np.zeros(DESCRIPTOR_SIZE_ONE_LEADER,dtype=float) )
            continue
        keys = A1.shape
        A2 = A1[follows, :]
        A3 = A2[:, follows]
        A = 1*A3
        keys = A.shape
        n = A.shape[0] ## number of followers
        mask = np.ones(old_n, dtype=bool)
        mask[leaders] = False
        B = A1[mask,:] #removing corresponding rows of leaders from laplacian
        B = B[:,leaders] #selecting corresponding columns of leaders from laplacian
        A = np.mat(A)
        B = np.mat(B)
        if INCLUDE_GRM:
            grm = controlpy.analysis.controllability_gramian(A,B)
            ranks = LA.matrix_rank(grm)
            traces = np.trace(grm) 
            if INCLUDE_EIGEN_VALUES :
                w, v = LA.eig(grm)
                a = np.real(w)
                a[a == 0] = 0.0001
                minval = np.min(a)
                min_eigs = np.min( minval )
                max_eigs = np.max( a ) 
            if INCLUDE_INVERSE_GRM :
                try:
                    grm = LA.pinv( grm ,hermitian=True)
                except:
                    print("Oops!", sys.exc_info()[0], "occurred.")
                    return None
                itraces = np.trace(grm)
                iranks =  LA.matrix_rank(grm)
                if INCLUDE_EIGEN_VALUES :
                    w, v = LA.eig(grm)
                    a = np.real(w)
                    a[a == 0] = 0.0001
                    
                    imin_eigs = np.min( a ) 
                    imax_eigs =  np.max( a )   

            if INCLUDE_METRIC_DIMENSION:            
                metric_dimension = compute_metric_dimension(G, leaders)

        new_desc.append(traces)
        new_desc.append(ranks)

        if INCLUDE_EIGEN_VALUES :
            new_desc.append(min_eigs)
            new_desc.append(max_eigs)
        if INCLUDE_INVERSE_GRM :   
            new_desc.append(itraces)
        if INCLUDE_EIGEN_VALUES and INCLUDE_INVERSE_GRM:
            new_desc.append( imin_eigs)
            new_desc.append(imax_eigs)

        if INCLUDE_METRIC_DIMENSION:
            new_desc.append(metric_dimension)

    n,m = G.number_of_nodes(),G.number_of_edges()
    new_desc.append(n)
    new_desc.append(m)
    new_desc.append(len(list(nx.biconnected_components(G))))
    if INCLUDE_LAP_SPECT :
        Ls = sorted(nx.laplacian_spectrum(G))
        Ls2 = Ls.copy()
        Ls2.extend( list(np.zeros(10) ) )
        new_desc.append(Ls2[0])
        new_desc.append(Ls2[1])
        new_desc.append(Ls2[2])
    if INCLUDE_LAP_SPECT_EXT :
        new_desc.append(Ls2[3])
        new_desc.append(Ls2[4])
        
        Ls2 = list(np.zeros(10) )
        Ls2.extend( Ls )
        
        new_desc.append(Ls2[-1])
        new_desc.append(Ls2[-2])
        new_desc.append(Ls2[-3])
        
    if INCLUDE_CYCLES:
        if n-m == 1:
            new_desc.append(1)  
        else:
            new_desc.append(0)
        if n-m == 0:
            new_desc.append(1)
        else:
            new_desc.append(0)
        if n-m == -1:
            new_desc.append(1)
        else:
            new_desc.append(0)
        if n-m < -1:
            new_desc.append(1)
        else:
            new_desc.append(0)
    if node_types==None:
        features = list(np.mean(d.x.numpy(),axis= 0))
        new_desc.extend(features)
    
    return new_desc

def add_vertex(G):
    G = nx.convert_node_labels_to_integers(G)
    nodes = sorted(list(G.nodes()))
    new_vertex = nodes[-1]+1 
    G.add_node(new_vertex)
    for n in G.nodes():
        if n !=new_vertex:
            G.add_edge(new_vertex, n)
    return G
def apply_RF_Grid(data_X,data_y):
    num_cores = multiprocessing.cpu_count()
    max_arr = []
    estimator  = RandomForestClassifier(criterion='gini', max_depth=None, min_weight_fraction_leaf=0.0,
                                            max_leaf_nodes=None, bootstrap=True, 
                                            oob_score=False, n_jobs=num_cores,verbose=0, warm_start=False,
                                            class_weight=None)
         
    param_grid = {'n_estimators':[50,100,500], 'max_features':['sqrt'], 
                  'min_samples_split':[2,3,4,5,10], 'min_samples_leaf':[1,2,5]}
    kf = StratifiedKFold(n_splits=10, random_state = 567, shuffle = True)
    grid_rf    = GridSearchCV(estimator, param_grid, scoring='accuracy', n_jobs=num_cores, 
                 refit=True, cv=kf, verbose=1, pre_dispatch='n_jobs', 
                 error_score='raise')

    grid_rf.fit(data_X, data_y)
    
    cv_results = pd.DataFrame(grid_rf.cv_results_)
    best_score = grid_rf.best_score_
    return best_score


def compute_metric_dimension(G, leaders):
    nodes_list = list(G.nodes()) 
    distance_vec = []
    for n in nodes_list:
        v_ = [nx.shortest_path_length(G, source=n, target=l) for l in leaders]
        distance_vec.append(v_)
    distance_vec = np.array(distance_vec)
    return np.unique(distance_vec, axis=0).shape[0]

def add_multiple_copies(G,atom_types):
    no_node = G.number_of_nodes()   
    H = G.copy()
    type_copy = atom_types.copy()
    H = nx.convert_node_labels_to_integers(H, ordering='sorted')
    val = sorted(G.nodes())
    val0 = val[0]
    while H.number_of_nodes()<10:
        val1 = H.number_of_nodes()-1
        H = nx.union(H, G, rename=('G1-', 'G2-'))
        atom_types.extend(type_copy)
        H.add_edge('G1-'+str(val1), 'G2-'+str(val0))
        H = nx.convert_node_labels_to_integers(H, ordering='sorted')
    return H, atom_types


In [ ]:

datasets = ["MUTAG","PTC_MR","ENZYMES"]
file = open("results/CTRL_bioinformatics_test.csv",'a',newline = '')
res_writer = csv.writer(file, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
res_writer.writerow(["dataset","CTRL_ACC","std","time"])
folds = 10
for f in datasets:
    accuracy = []
    for i in range(folds):
        attr = False
        if f=="ENZYMES":
            graph_dataset = TUDataset(root = "../data/TUDataset", name=f,use_node_attr = True)
            ALL_ELEMENTS = [0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20]
            node_types = None
        else:
            graph_dataset = TUDataset(root = "../data/TUDataset", name=f,use_node_attr = False)
            node_types = pickle.load(open("mapping_"+f+".pkl","rb"))
            ALL_ELEMENTS = [ 'O', 'N', 'C', 'S', 'F', 'Cl', 'P', 'Br', 'I', 'Si', 'Se', 'As', 'Pt', 
                    'B', 'Bi', 'Ga', 'Sn', 'Pb', 'Cu','VNode']
        start = time.time()
        control_des = Parallel(n_jobs=-1, verbose=2)(delayed(get_descriptor)(d, node_types, ALL_ELEMENTS) for d in graph_dataset)
        end = time.time()
        labels = [x.y.item() for x in graph_dataset]
#         np.save("data/CTRL_new_data/"+f+"_desc.npy", control_des)
        control_des = np.array(control_des)

        labels = np.array(labels)

        acc = apply_RF_Grid(control_des, labels)
        print(acc)
        accuracy.append(acc)
    to_write = [f,np.mean(accuracy)*100,np.std(accuracy)*100]
    res_writer.writerow(to_write)
    file.flush()
    print("results:", to_write)
#     break
file.close()
    